# 数据清洗

现实中的数据总有各种各样的错误，称为脏（dirty）数据或坏数据。坏数据在数据分析中可能会引发千奇百怪的异常与错误，所以有人说，坏数据要比无数据还坏。故在数据分析之前，常常要花费大量时间来清洗数据，来处理这些脏数据或坏数据。

对于杂乱无规则的脏数据，常常可以归类如下：
- 无用数据
- 重复数据
- 缺失数据
- 错误或异常数据

本节介绍相应的数据清洗方法。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 无用数据

顾名思义，加载读取的数据集中有些字段与所需无关。对应的数据清洗方法有二：
- 保留下来不理它，只是耗些内存而已。
- 删除它，眼不见心不烦。

### 删除无用数据

使用`df.drop()`方法来删除无用数据：

In [ ]:
df = pd.DataFrame(np.arange(24).reshape(3, 8),
                  columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
df

In [ ]:
to_drop = ['B', 'D', 'F', 'H']
df.drop(to_drop, axis=1, inplace=True)
df

### 使用列选择

使用反向选择的方法：

In [ ]:
df = pd.DataFrame(np.arange(24).reshape(3, 8),
                  columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
df

In [ ]:
colnames = ['A', 'C', 'E', 'G']
df = df[colnames]
df

## 重复数据

Pandas 数据帧中的数据行是可以重复的，判断数据是否重复取决于数据的实际意义。在数据收集过程中，特别是人工收集的过程中，常会出现数据重复的现象。

### 重复数据检查

`df.duplicated()`用来检查数据是否重复，其使用语法为：
```python
df.duplicated(subset=None, keep='first')
```
主要参数说明：
- `subset=None`，识别重复标签（字符串或列表），默认是所有列标签；
- `keep='first'`：`'first'`标记第一个之外数据；`'last'`标记最后一个之外数据；`False`标记全部重复数据；

In [ ]:
df = pd.DataFrame({
    'id1': ['a', 'a', 'a', 'b', 'b', 'b'],
    'id2': ['a1', 'a2', 'a2', 'b1', 'b2', 'b2'],
    'x': [1, 2, 3, 4, 5, 5] ,
    'y': [0.1, 0.3, 0.3, 0.4, 0.5, 0.5] ,
})

df

In [ ]:
# 标记所有数据
df.duplicated(keep=False)

In [ ]:
# 标记最后一个之外数据
df.duplicated(['id1', 'id2'], keep='last')

### 数据去重

使用 `df.drop_duplicates()` 方法用来删除重复数据，即数据去重，其使用语法为：
```python
df.drop_duplicates(subset=None, keep='first', inplace=False)
```
主要参数说明：
- `subset=None`，识别重复标签（标量或序列），默认是所有列标签；
- `keep='first'`：`'first'`表示从前向后，后面相同数据是重复；`'last'`表示从后向前，之前相同数据是重复值；`False`标记全部重复数据；
- `inplace=False`，是否在对象中直接修改；

下面代码返回数据去重后的新对象：

In [ ]:
df.drop_duplicates(keep='first')

In [ ]:
df.drop_duplicates(['id1', 'id2'], keep='first')

在实际工作中，有时会先对数据进行排序，然后再进行去重，以保留自己实际需要数据。

假定一个数据集，是对于不同id的事物，在不同时间进行测量得到一个结果：

In [ ]:
df = pd.DataFrame({
    'id': ['a', 'a', 'c', 'b', 'b', 'b'],
    'date': ['2018-01-01', '2018-10-01', '2018-09-01', '2018-01-01', '2018-10-01', '2018-10-01'],
    'value': [1, 2, 3, 4, 5, 5] ,
})
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

如果只想获得最近的测量结果，可以对时间列进行排序再去重即可：

In [ ]:
df.sort_values(['date'], ascending=[False], inplace=True)
df

In [ ]:
df_droped = df.drop_duplicates(['id'])
df_droped

## 缺失数据

在实际工作中，由于种种原因，采集到的数据中会存在缺失。Pandas设计目标之一就是便利地处理缺失值。然而，了解数据的缺失以及对缺失数据进行处理，仍然是一件重要的事。

仍然使用上一示例数据：

In [ ]:
df = pd.DataFrame({
    'id': ['a', 'a', 'c', 'b', 'b', 'b'],
    'date': ['2018-01-01', '2018-10-01', '2018-09-01', '', '2018-10-01', '2018-10-01'],
    'value': [1, 2, 3, np.nan, 5, 5] ,
})

df['date'] = pd.to_datetime(df['date'])

df.info()

In [ ]:
df.head()

在Pandas中，对于数值型数据，使用`NaN (Not a Number)`来表示缺失值；对于日期型`datetime64`则使用`NaT (Not a Time)`来表示。Pandas 对缺失数据的处理方法有如下：
- `df.isnull()`: 返回缺失数据的布尔结果
- `df.notnull()`：`isnull()`的否定式
- `df.dropna()`：去除缺失数据
- `df.fillna()`：填充缺失数据

### 缺失值检查

使用 `df.isnull()` 与 `df.notnull()` 来判断数据中是否存在缺失值：

In [ ]:
# 检查所有列
df.isnull()

In [ ]:
# 检查指定列
df['date'].isnull()

In [ ]:
df['value'].notnull()

### 缺失值删除

使用 `df.dropna()` 删除缺失值，其使用语法为：
```python
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
```
主要参数说明：
- `axis=0`：默认是丢弃行;
- `how='any`: 默认是丢弃任何含有缺失值的行，`all`表示全部是缺失值才丢弃;

In [ ]:
df.dropna(how='any')
# df.dropna(how='all')

### 缺失值填充

使用`df.fillna()`方法来补全缺失值，而非简单过滤，其使用语法为：
```python
df.fillna(value=None, method=None, axis=None, inplace=False, limit=None, downcast=None, **kwargs)
```
主要参数：
- `value`： 指定标量或字典对象用于填充缺失值。
- `method`： 指定插值方法，默认是`ffill`前填充。

In [ ]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                  columns=list('ABCD'))
df

下面使用指定值来填充：

In [ ]:
df.fillna(0)

使用字典型对象，不同的列用不同的值来填充：

In [ ]:
values = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
df.fillna(value=values)

使用插值方法来填充，插值方法包括：
- `ffill`，用前一行的数据来填充。
- `bfill`，后后一行的数据来填充。

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.fillna(method='bfill')

对于数值型列，会用一些统计值，例如均值、中位数等来填充缺失值。

## 错误和异常数据

### 错误数据替换
在数据集中，有些数据存在明显错误，需要进行清洗或者修正。使用 `replace()` 方法可以快捷进行修改，其使用语法为：
```python
df.replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad')
s.replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad')
```

使用 `to_replace` 与 `value`，使用直接指定方法实现数据替换。

指定`regex=True`，使用正则表达式来实现数据替换。

旧值与替换值都为标量：

In [ ]:
# 0 ==>  5
s = pd.Series([0, 1, 2, 3, 4])
s.replace(0, 5)

旧值为列表，替换值为标量或列表：

In [ ]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'c', 'd', 'e']})
df

In [ ]:
# [0, 1, 2, 3] => 4
df.replace([0, 1, 2, 3], 4)

In [ ]:
# [0, 1, 2, 3] => [4, 3, 2, 1]
df.replace([0, 1, 2, 3], [4, 3, 2, 1])

使用字典方式

In [ ]:
# 0 in A， 5 in B  => 100
df.replace({'A': 0, 'B': 5}, 100)

指定`regex=True`，则使用正则表达式方法来进行替换，其它则类似：

In [ ]:
df = pd.DataFrame({'A': ['bat', 'foo', 'bait'],
                   'B': ['abc', 'bar', 'xyz']})
df.replace(to_replace=r'^ba.$', value='new', regex=True)

### 异常数据清洗

在数据集中，还有一些数据的格式正常，但其值超乎常规或者不符合业务逻辑。对于这些“异常”数据，要特别小心。它们或许是错误的，但也可能是真实的。

这些异常数据，通常要结合业务逻辑来判断。不同数据集其方法也不同。常用方法是先使用统计得到标准房差，然后设定阈值，超过多少倍标准方差的即为异常数据。

考虑生成一个IT与金融行业工资收入的随机分布数据集：

In [ ]:
incomes = pd.DataFrame({
    'it': np.random.normal(50000, 25000, 1024),
    'finance': np.random.normal(80000, 25000, 1024)
})
incomes.head()

添加几个异常值：

In [ ]:
incomes['it'][512] = 1000000
incomes['finance'][256] = 20000000

可以得到它们的统计结果：

In [ ]:
incomes.describe()

设定8倍标准方差的数据为异常数据：

In [ ]:
incomes_sigma = incomes.std()
incomes_sigma

In [ ]:
incomes.loc[abs(incomes['it']) > 8 * incomes_sigma['it'], 'it']

In [ ]:
incomes.loc[abs(incomes['finance']) > 8 * incomes_sigma['finance'], 'finance']